In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from myMLlib import set_seed, load_feather
import yaml
from sklearn.preprocessing import StandardScaler
import types

In [2]:
import cca_zoo
print(cca_zoo.__version__)
import sklearn
print(sklearn.__version__)

2.6.0
1.7.2


In [3]:
folderpath= ('/home/yuqster/files/research/dataset/realtest1',
             '/home/yuqster/files/research/dataset/realtest2',
             '/home/yuqster/files/research/dataset/realtest3',
             '/home/yuqster/files/research/dataset/realtest4',
             '/home/yuqster/files/research/dataset/realtest5',
             )
plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 20,
    'axes.labelsize': 16,
    'xtick.labelsize': 18,
    'ytick.labelsize': 18,
    'legend.fontsize': 14
})
with open('hyperparameters.yml', 'r') as file:
    all_parameters = yaml.safe_load(file)
    parameters = all_parameters['real-blood-meat-time']
materials = parameters['materials']
realmater = parameters['realmater']
materials = list(materials.values())
print(materials)
print(realmater)

['Vessel', 'Tissue']
['Silicone/Water', 'Silicone']


In [4]:
seed = 43
set_seed(seed)
data_list,labels = load_feather(folderpath,materials,[1e4 , 1e4, 1e4, 1e4, 1e4], rand=True)
X_list = [np.array(X).squeeze() for X in data_list]

In [5]:
print(X_list[1].shape)
X_list = [np.array(X).squeeze() for X in data_list]

(10000, 4096)


In [7]:
def multi_SA_memwise(Ulist, r=None, weights=None):
    Ucut = [U[:, : (r or min(U.shape[1] for U in Ulist))] for U in Ulist]
    Ustack = np.stack(Ucut)                          # (m,d,r)
    if weights is None:
        # C[d,e] = sum_{m,r} U[m,d,r] * U[m,e,r]
        C = np.tensordot(Ustack, Ustack, axes=([0,2],[0,2]))   # (d,d)
    else:
        w = np.asarray(weights, float)[:, None, None]           # (m,1,1)
        C = np.tensordot(w * Ustack, Ustack, axes=([0,2],[0,2]))
    eigvals, eigvecs = np.linalg.eigh(C)
    idx = np.argsort(eigvals)[::-1][:Ustack.shape[2]]
    return eigvecs[:, idx]                          # U_shared
